In [ ]:
%pip install --upgrade pip
%pip install matplotlib
%pip install osmnx
%pip install networkx
%pip install csv
%pip install scikit-learn


In [ ]:

import networkx as nx
import osmnx as ox
import numpy as np
import time
import matplotlib.pyplot as plt
import csv
import random
from dijkstra_algorithm import dijkstra_algorithm
from dijkstra_algorithm import make_route
from dijkstra_algorithm import dijkstra_algorithm_driver
from deliveries import delivery
from other_funcs import *
from minheap import MinHeap
from maxheap import MaxHeap
from driver import driver
import heapq

ox.__version__
random.seed(None)

In [ ]:
# download/model a street network for some city then visualize it
G = ox.graph_from_place("Chicago, Illinois, USA", network_type="drive")
Gc = ox.utils_graph.get_largest_component(G, strongly=True)

Need to make a list of all restaurants using what is below this markdown

In [ ]:
file = open('Food_Inspections_20240223.csv')
type(file)

csvreader = csv.reader(file)
header = []
header = next(csvreader)
header

food_inspections = []
filled_locations = []

rows = []
for row in csvreader:
    rows.append(row)

I condensed all the sections into a single function that does the graphing.

In [ ]:
def get_graph(num_of_drivers,num_of_deliveries,graph,restaurants):
    N_L = list(graph.nodes)[None:None:None]
    graph = ox.speed.add_edge_speeds(graph)
    graph = ox.speed.add_edge_travel_times(graph)
    items = random_restaurant_locations(restaurants,num_of_deliveries)
    list_nodes = find_node(items,graph)

    list_of_deliveries = []
    for i in list_nodes:
        d = delivery(i,N_L)
        list_of_deliveries.append(d)
    
    drivers = []
    for i in range(num_of_drivers):
        single = driver(N_L)
        drivers.append(single)
    find_routes_for_res_to_dropoff(graph,list_of_deliveries)
    restaurant_set = set()
    for i in list_of_deliveries:
        restaurant_set.add(i.restaurant_loc)
    
    find_shortest_driver_dikstra(graph,list_of_deliveries,drivers,restaurant_set)

    total = []
    for i in list_of_deliveries:
        total.append(i.reward_ratio[i.driver_loc])
    max_reward = max(total)
    average = sum(total) / len(total)
    return max_reward,average
    

In [ ]:
def generate_random_input(size):
    arr = np.random.randint(0, 100000, size = size)
    return arr

In [ ]:
monte_carlo_runs = 30


# Range of array sizes considered
start_array = 2
end_array = 20
increments = 2

# Initiate average and worst case run-time
avg_rewards_earned = [0]*(end_array//increments)
best_rewards = [0]*(end_array//increments)

# Run-time experiments for random arrays of a given size
count = 0
for curr_array_size in range(start_array, end_array+1, increments):  
    average_rewards = []
    max_rewards = []
    # Given a fixed array size, generate one random array and compute its run-time
    for iteration in range(monte_carlo_runs):             
        max_reward, average = get_graph(curr_array_size,curr_array_size,Gc,rows)                    
        average_rewards.append(average)
        max_rewards.append(max_reward)           
        
    avg_rewards_earned[count] = (sum(average_rewards)/len(average_rewards)) # Empirical average of run-time
    best_rewards[count] = (max(max_rewards))              # Worst case run time across various random experiments
    count += 1

In [ ]:
# Data Coordinates
x = np.arange(start_array, end_array+1, increments)
y = np.array(avg_rewards_earned)
z = np.array(best_rewards)

# Plot
plt.plot(x,y,'b',label='Avg Reward Earned') 
plt.plot(x,z,'r',label='Best Reward earned') 

# Add Title
plt.title("Delivery Simulation") 

# Add Axes Labels
plt.xlabel("# of delveries") 
plt.ylabel("Reward Ratio") 

# Display
plt.legend()
plt.show()